In [ ]:
! pip install pypdf
! pip install -q transformers einops accelerate langchain bitsandbytes
pip install -U langchain-community
! pip install sentence_transformers
! pip install llama_index
!pip install llama-index-llms-huggingface

In [ ]:
import os
import torch
from sentence_transformers import SentenceTransformer
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core import ServiceContext
from llama_index.legacy.embeddings.langchain import LangchainEmbedding


In [ ]:
!!huggingface-cli login

Load PDF files

In [ ]:
# Function to recursively read all documents in a directory
def load_all_documents(parent_directory):
    all_documents = []
    for root, dirs, files in os.walk(parent_directory):
        if files:
            reader = SimpleDirectoryReader(root)
            documents = reader.load_data()
            all_documents.extend(documents)
    return all_documents

# Define the parent directory containing multiple folders
parent_directory = "/content/data"

# Load all documents from the parent directory
documents = load_all_documents(parent_directory)

In [ ]:
system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided.
"""
#default format supportable by LLama2
query_wrapper_prompt=SimpleInputPrompt("<|USER|> {query_str}<|ASSISTANT|>")

In [ ]:
llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature":0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    model_kwargs={"torch_dtype": torch.float16, "load_in_8bit":True}

)
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))



Create vector database and store

In [ ]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)
index = VectorStoreIndex.from_documents(documents, service_context=service_context)
index.storage_context.persist()

In [ ]:
query_engine = index.as_query_engine()

Test examples

In [ ]:
response = query_engine.query("What are the OTC medication for shingle")
print(response)

In [ ]:
response = query_engine.query("Provide some products with brand name, price, ingredients for dermatsis?")
print(response)

In [ ]:
response = query_engine.query("Provide the best product for hives? List products recommendation i can buy for either home treatment or drug choice")
print(response)